# CourtListener

In [ ]:
import pandas as pd
import json
import re

## Type O

In [18]:
# Read the JSONL file
data_o_path = 'courtlistener/search/o/2025-06-13_courtlistener_search_type-o_artificial-intelligence.jsonl'

with open(data_o_path, 'r') as file:
    data_o = [json.loads(line) for line in file]

In [ ]:
# Transform the data into a DataFrame
dataset_o = pd.DataFrame(data_o)

dataset_o.head()

,absolute_url,attorney,caseName,caseNameFull,citation,citeCount,cluster_id,court,court_citation_string,court_id,...,panel_ids,panel_names,posture,procedural_history,scdb_id,sibling_ids,source,status,suitNature,syllabus
0,/opinion/6520938/noerr-v-lewistown-smelting-re...,"Albert Houck, for plaintiff., Houck & Barron, ...","Noerr v. Lewistown Smelting & Refining, Inc.","Noerr v. Lewistown Smelting & Refining, Inc.","[60 Pa. D. & C.2d 406, 1973 Pa. Dist. & Cnty. ...",0,6520938,"Pennsylvania Court of Common Pleas, Mifflin Co...",,pactcomplmiffli,...,[],[],,,,[6394599],U,Published,,
1,/opinion/7372834/sharp-v-stalker/,"Mr. Alan TL. Strong, for the complainants., Mr...",Sharp v. Stalker,Willoughby W. Sharp v. John Stalker et ux.,"[63 N.J. Eq. 596, 18 Dickinson 596, 52 A. 1120...",0,7372834,New York Court of Chancery,,nychanct,...,[],[],,,,[7291653],U,Published,,
2,/opinion/3438130/shutes-v-weeks/,"Stipp, Perry, Bannister & Starzinger and Putna...",Shutes v. Weeks,"Marguerite Shutes Et Al., Administrators, Appe...","[262 N.W. 518, 220 Iowa 616]",25,3438130,Supreme Court of Iowa,Iowa,iowa,...,[],[],Appeal from Polk District Court. &#8212; JOSEP...,,,[3436026],ZU,Published,,
3,/opinion/7983134/osten-v-jerome/,"Eldredge & Spier, for appellant., James G. Tuc...",Osten v. Jerome,Charles Osten v. Edwin Jerome,"[93 Mich. 196, 1892 Mich. LEXIS 964, 53 N.W. 7]",0,7983134,Michigan Supreme Court,Mich.,mich,...,[],[],,,,[7936034],U,Published,,
4,/opinion/6757835/alabama-v-acacia-mut-life-assn/,"Chilton & McCoy, of Montgomery, Ala., for the ...",Alabama v. Acacia Mut. Life Ass'n,STATE OF ALABAMA v. ACACIA MUT. LIFE ASS'N,[3 F.2d 697],0,6757835,"District Court, M.D. Alabama",M.D. Ala.,almd,...,[],[],,,,[6640454],U,Published,,


In [ ]:
print(f'Fields: {list(dataset_o.columns)}')
dataset_o.info()

Fields: ['absolute_url', 'attorney', 'caseName', 'caseNameFull', 'citation', 'citeCount', 'cluster_id', 'court', 'court_citation_string', 'court_id', 'dateArgued', 'dateFiled', 'dateReargued', 'dateReargumentDenied', 'docketNumber', 'docket_id', 'judge', 'lexisCite', 'meta', 'neutralCite', 'non_participating_judge_ids', 'opinions', 'panel_ids', 'panel_names', 'posture', 'procedural_history', 'scdb_id', 'sibling_ids', 'source', 'status', 'suitNature', 'syllabus']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6913 entries, 0 to 6912
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   absolute_url                 6913 non-null   object
 1   attorney                     6913 non-null   object
 2   caseName                     6913 non-null   object
 3   caseNameFull                 6913 non-null   object
 4   citation                     6913 non-null   object
 5   citeCount             

In [ ]:
# Check for missing values in the dataset, especially 'caseNameFull'
print("Missing values in each column:")

dataset_o.isna().sum()

Missing values in each column:


absolute_url                      0
attorney                          0
caseName                          0
caseNameFull                      0
citation                          0
citeCount                         0
cluster_id                        0
court                             0
court_citation_string             0
court_id                          0
dateArgued                     6799
dateFiled                         0
dateReargued                   6909
dateReargumentDenied           6828
docketNumber                     99
docket_id                         0
judge                             0
lexisCite                         0
meta                              0
neutralCite                       0
non_participating_judge_ids       0
opinions                          0
panel_ids                         0
panel_names                       0
posture                           0
procedural_history                0
scdb_id                           0
sibling_ids                 

In [ ]:
# pattern catches ai or artificial intelligence or arificial-intelligence
pattern = r'\b(ai|artificial[\s\-]?intelligence)\b'

ai_cases_o = dataset_o[dataset_o['caseNameFull'].str.contains(pattern, case=False, na=False, regex=True)]


/var/folders/1v/4gp6708s7gg9znljvp5fhtwh0000gn/T/ipykernel_14781/329156998.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ai_cases = dataset[dataset['caseNameFull'].str.contains(pattern, case=False, na=False, regex=True)]


In [ ]:
ai_cases_o

,absolute_url,attorney,caseName,caseNameFull,citation,citeCount,cluster_id,court,court_citation_string,court_id,...,panel_ids,panel_names,posture,procedural_history,scdb_id,sibling_ids,source,status,suitNature,syllabus
1923,/opinion/1933074/artificial-intelligence-corp-...,"John W. Cutchin and Thomas R. Darton, Duckor, ...",Artificial Intelligence Corp. v. Casey (In Re ...,"In Re Thomas F. CASEY, Debtor. ARTIFICIAL INTE...","[193 B.R. 942, 1996 WL 137293]",3,1933074,"United States Bankruptcy Court, S.D. California",Bankr. S.D. Cal.,casb,...,[],[],,,,[1933074],LU,Published,,
1924,/opinion/1925615/artificial-intelligence-corp-...,"Roy L. Carlson, Jr., Milberg & De Phillips, P....",Artificial Intelligence Corp. v. Casey (In Re ...,"In Re Thomas F. CASEY, Debtor. ARTIFICIAL INTE...","[198 B.R. 918, 1996 Bankr. LEXIS 919, 1996 WL ...",6,1925615,"United States Bankruptcy Court, S.D. California",Bankr. S.D. Cal.,casb,...,[],[],,,,[1925615],LU,Published,,
1927,/opinion/2000137/audre-recognition-systems-inc...,"Alan Van Derhoff, Sheppard, Mullin, Richter & ...",Audre Recognition Systems Inc. v. Casey (In Re...,"In Re AUDRE, INC., a California Corporation, D...","[210 B.R. 360, 38 Collier Bankr. Cas. 2d 500, ...",4,2000137,"United States Bankruptcy Court, S.D. California",Bankr. S.D. Cal.,casb,...,[],[],,,,[2000137],LU,Published,,
3352,/opinion/7321124/loop-ai-labs-inc-v-gatti/,"Bryan Jacob Wolin, Diana Wong, Valeria Calafio...",Loop AI Labs Inc. v. Gatti,LOOP AI LABS INC. v. Anna GATTI,"[195 F. Supp. 3d 1107, 2016 U.S. Dist. LEXIS 8...",6,7321124,"District Court, N.D. California",N.D. Cal.,cand,...,[],[],,,,[7239034],U,Published,,
